# CROHME 2023

## Imports

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import keras
from keras import layers
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))


2024-12-16 14:13:16.918511: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-16 14:13:16.928910: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734387196.941795   44768 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734387196.944924   44768 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-16 14:13:16.957468: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Num GPUs Available:  1


Import `crohme_dataset`

In [2]:
import datasets.crohme_dataset  # Register `crohme_dataset`

ds = tfds.load("crohme_dataset")  # `crohme_dataset` registered
test: tf.data.Dataset = ds["test"]
train: tf.data.Dataset = ds["train"]
validation: tf.data.Dataset = ds["validation"]

I0000 00:00:1734387200.309734   44768 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4768 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


### Extra: Previewing InkML Files

I also created a little utility in C++ and GTK to render out an inkml file from the dataset. It reads the InkML file, and renders out the strokes as well as the LaTeX of what it's supposed to be. It was a fun project!

In [3]:
import os

random_data_point = next(iter(validation.shuffle(200_000).take(1)))
filepath = random_data_point["filepath"].numpy().decode("ascii")
os.system(f"inkmlviewer {filepath}")

2024-12-16 14:13:22.071454: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:376] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608


/home/jeshwinprince/Programming/crohme/datasets/crohme_dataset/data/INKML/val/CROHME2023_val/form_061_E488.inkml
Displaying app now...


0

## Preprocessing

### Text Vectorization

We will use `pylatexenc` to parse the LaTeX into nodes for custom splitting

In [33]:
from pylatexenc.latexwalker import (
    LatexWalker,
    LatexMacroNode,
    LatexEnvironmentNode,
    LatexCharsNode,
    LatexGroupNode,
)

START_TOKEN, END_TOKEN = "<START>", "<END>"


# Define the tokenization function using pylatexenc
def latex_tokenizer(latex_string):
    """
    Tokenizes a LaTeX string into tokens using pylatexenc.
    """
    if not latex_string:
        return []
    walker = LatexWalker(latex_string)

    def parse_node(nodelist):
        if len(nodelist) == 0:
            return []
        try:
            tokens = []
            for node in nodelist:
                if not node:
                    continue
                elif node.isNodeType(LatexMacroNode):
                    tokens.append(f"\\{node.macroname}")
                    # Parse arguments if they exist
                    tokens += parse_node(node.nodeargd.argnlist)
                elif node.isNodeType(LatexEnvironmentNode):
                    tokens.append(f"\\begin{{{node.environmentname}}}")
                    tokens += parse_node(node.nodeargd.argnlist)
                    tokens += parse_node(node.nodelist)
                    tokens.append(f"\\end{{{node.environmentname}}}")
                elif node.isNodeType(LatexCharsNode):
                    tokens += list(node.chars)
                elif node.isNodeType(LatexGroupNode):
                    tokens.append(node.delimiters[0])
                    tokens += parse_node(node.nodelist)
                    tokens.append(node.delimiters[1])
            return tokens
        except Exception as e:
            return []

    nodelist, _, _ = walker.get_latex_nodes()
    return parse_node(nodelist)


# Wrap the tokenizer for use in TextVectorization
def tokenize_fn(latex_tensor):
    tokens = []
    for latex_string in latex_tensor:
        tokenized_string = latex_tokenizer(latex_string.numpy().decode("utf-8"))
        tokenized_string.insert(0, START_TOKEN)
        tokenized_string.append(END_TOKEN)
        tokens.append(tokenized_string)
    return tf.ragged.constant(tokens, dtype=tf.string)


# Create a TensorFlow-compatible wrapper
def tf_tokenizer(latex_string):
    return tf.py_function(
        func=tokenize_fn,
        inp=[latex_string],
        Tout=tf.RaggedTensorSpec([None, None], dtype=tf.string),
    )

Create the vectorizer and use a vocabulary file to adapt it

In [ ]:
# Create the TextVectorization layer
max_tokens = 10_000  # Adjust depending on your vocabulary size

vectorizer = layers.TextVectorization(
    max_tokens=max_tokens,
    standardize=None,  # Custom tokenizer, so no built-in preprocessing
    split=tf_tokenizer,
    ragged=True,
)
dataset = tf.data.TextLineDataset("vocabulary.txt")
dataset = dataset.map(lambda line: [line])
vectorizer.adapt(dataset)

Test it out to make sure it works properly

In [ ]:
def latex_to_token(string):
    return vectorizer(string)


id_to_token = {i: token for i, token in enumerate(vectorizer.get_vocabulary())}


def token_to_latex(tokens):
    return "".join([id_to_token[id] for id in tokens.numpy()])


latex_array = [
    r"E = mc^2",
    r"\frac{a}{b} + \sqrt{c}",
    r"\sum_{i=1}^n i^2 = \frac{n(n+1)(2n+1)}{6}",
    r"A = \pi r^2",
    r"G=\begin{bmatrix}1&\dots&1&0&\dots&0\\ \ast&\ast&\ast&&G^{\prime}&\\ \end{bmatrix}",
]
latex_data = tf.constant(latex_array)

# Tokenize and vectorize
tokenized_output = latex_to_token(latex_data)
print(tokenized_output)

E_mc2 = token_to_latex(tokenized_output[0])
print(E_mc2)

Save the vocabulary just in case. You would still need the custom latex parser function imported to use it in another project

In [ ]:
with open("vectorizer_vocabulary.txt", "w") as f:
    for word in vectorizer.get_vocabulary():
        f.write(word)
        f.write("\n")

To re-create the vectorizer using the vocabulary, run the following code

In [34]:
max_tokens = 10_000
vectorizer = layers.TextVectorization(
    max_tokens=max_tokens,
    standardize=None,  # Custom tokenizer, so no built-in preprocessing
    split=tf_tokenizer,
    ragged=True,
)
with open("vectorizer_vocabulary.txt", "r") as f:
    lines = [line[:-1] for line in f]
    vectorizer.set_vocabulary(lines)

Test it out to make sure it works properly

In [35]:
def latex_to_token(string):
    return vectorizer(string)


id_to_token = {i: token for i, token in enumerate(vectorizer.get_vocabulary())}


def token_to_latex(tokens):
    return "".join([id_to_token[id] for id in tokens.numpy()])


latex_array = [
    r"E = mc^2",
    r"\frac{a}{b} + \sqrt{c}",
    r"\sum_{i=1}^n i^2 = \frac{n(n+1)(2n+1)}{6}",
    r"A = \pi r^2",
    r"G=\begin{bmatrix}1&\dots&1&0&\dots&0\\ \ast&\ast&\ast&&G^{\prime}&\\ \end{bmatrix}",
]
latex_data = tf.constant(latex_array)

# Tokenize and vectorize
tokenized_output = latex_to_token(latex_data)
print(tokenized_output)

E_mc2 = token_to_latex(tokenized_output[0])
print(E_mc2)

<tf.RaggedTensor [[4, 60, 45, 11, 45, 30, 39, 9, 12, 5],
 [4, 20, 3, 21, 2, 3, 38, 2, 45, 18, 45, 79, 3, 39, 2, 5],
 [4, 73, 6, 3, 19, 11, 10, 2, 9, 16, 45, 19, 9, 12, 45, 11, 45, 20, 3, 16,
  7, 16, 18, 10, 8, 7, 12, 16, 18, 10, 8, 2, 3, 92, 2, 5]                 ,
 [4, 34, 45, 11, 45, 78, 27, 9, 12, 5],
 [4, 76, 11, 165, 10, 154, 10, 17, 154, 17, 75, 45, 245, 245, 245, 76, 9, 3,
  69, 2, 75, 45, 164, 5]                                                    ]>
<START>E = mc^2<END>


Try an actual data point

In [36]:
random_data_point = next(iter(validation.take(1)))
tokenized_data_point = latex_to_token([random_data_point['ground_truth'].numpy()])
print(tokenized_data_point)
detokenized_data_point = token_to_latex(tokenized_data_point[0])
print(detokenized_data_point)

<tf.RaggedTensor [[4, 15, 10, 29, 127, 77, 77, 5]]>
<START>-1.955<END>


### Preprocessing Strokes

Instead of images, this model takes in a stream of strokes, such as writing with a stylus on a tablet. Our dataset gives us a list of strokes, and each stroke is itself a list of coordinates [x, y] of the position of the stylus. Both the number of strokes and the length of each strokes changes for every value in our dataset, so we are going to pre-process the stroke data so it will be normalized (scaled to be between 0 and 1), and always fit in a tensor with shape `(64, 64, 2,)`. FOr this, I am using the [Ramer-Douglas-Peucker Algorithm](https://en.wikipedia.org/wiki/Ramer%E2%80%93Douglas%E2%80%93Peucker_algorithm) for polyline decimation.

In [123]:
@tf.function
def preprocess_strokes(strokes: tf.RaggedTensor):
    # First, scale values to between 0.0 and 1.0
    min_vals = tf.reduce_min(strokes, axis=(0, 1))
    max_vals = tf.reduce_max(strokes, axis=(0, 1))
    normalized_strokes = tf.map_fn(
        elems=strokes,
        fn=lambda stroke: (stroke - min_vals) / (max_vals - min_vals + 1e-6),
    )

    def point_line_distance(point, start, end):
        """
        Calculate the perpendicular distance from `point` to the line segment
        defined by `start` and `end`.
        """
        # Convert to 3D by adding a zero z-component
        point_3d = tf.concat([point, tf.zeros([1], dtype=tf.float32)], axis=0)
        start_3d = tf.concat([start, tf.zeros([1], dtype=tf.float32)], axis=0)
        end_3d = tf.concat([end, tf.zeros([1], dtype=tf.float32)], axis=0)

        # Return the perpendicular distance (norm of the cross product / norm of the line segment)
        return tf.norm(
            tf.linalg.cross(end_3d - start_3d, point_3d - start_3d)
        ) / tf.norm(end_3d - start_3d)

    def douglas_peucker(stroke, epsilon=0.02):
        """
        Non-recursive Douglas-Peucker algorithm implementation.
        """

        # Initialize the list of points to keep
        simplified_stroke: tf.TensorArray = tf.TensorArray(
            dtype=stroke.dtype, size=0, dynamic_size=True
        ).write(0, stroke[0])

        # Stack for processing: Each entry contains a tuple (start_index, end_index)
        stack: tf.TensorArray = tf.TensorArray(
            dtype=tf.int32, size=0, dynamic_size=True
        ).write(0, [0, tf.shape(stroke)[0] - 1])

        def cond(stack: tf.TensorArray, _simplified_stroke: tf.TensorArray):
            return tf.not_equal(stack.size(), 0)

        def body(stack: tf.TensorArray, simplified_stroke: tf.TensorArray):
            # Pop from the stack
            border_idxs = stack.read(stack.size() - 1)
            start_idx, end_idx = border_idxs[0], border_idxs[1]

            if stack.size() > 1:
                new_stack_tensor = stack.gather(tf.range(stack.size() - 1))
                # Create a new stack with the remaining elements
                new_stack = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
                for i in tf.range(stack.size() - 1):
                    new_stack = new_stack.write(i, new_stack_tensor[i])
                stack = new_stack
            else:
                stack = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)

            # Get the relevant slice of the stroke
            sub_stroke = stroke[start_idx : end_idx + 1]

            # Calculate the perpendicular distances of all intermediate points
            start, end = sub_stroke[0], sub_stroke[-1]
            distances = tf.vectorized_map(
                elems=sub_stroke[1:-1], fn=lambda p: point_line_distance(p, start, end)
            )

            if tf.size(distances) > 0:
                max_distance = tf.reduce_max(distances)
                max_idx = (
                    tf.argmax(distances, output_type=tf.int32) + 1
                )  # +1 because we skip the start point

                # If the max distance is greater than epsilon, continue splitting
                if max_distance > epsilon:
                    stack = stack.write(stack.size(), [start_idx, start_idx + max_idx])
                    stack = stack.write(stack.size(), [start_idx + max_idx, end_idx])
                else:
                    # Otherwise, keep the start and end points
                    simplified_stroke = simplified_stroke.write(
                        simplified_stroke.size(), end
                    )
            else:
                # If no intermediate points exist, just keep the start and end points
                simplified_stroke = simplified_stroke.write(
                    simplified_stroke.size(), end
                )

            return stack, simplified_stroke

        # Return the simplified stroke
        return tf.while_loop(cond, body, [stack, simplified_stroke])[1].stack()

    downsampled_strokes = tf.map_fn(elems=normalized_strokes, fn=douglas_peucker)
    return downsampled_strokes

Let's try it out first!

In [124]:
random_data_point = next(iter(validation.shuffle(100_000).take(1)))
# print(random_data_point['strokes'])
print(preprocess_strokes(random_data_point["strokes"]))

<tf.RaggedTensor [[[0.0014870232, 0.6382595],
  [0.9724908, 0.46872157],
  [0.6565056, 0.48866728]]  , [[0.3999999, 0.10335445],
                               [0.5702602, 0.2203083],
                               [0.594052, 0.10154082],
                               [0.57174724, 0.0]]      ,
 [[0.15092932, 0.7969177],
  [0.13605936, 0.98458767],
  [0.21933068, 0.96192217],
  [0.2802975, 0.849501],
  [0.24609652, 0.8059837]] , [[0.3754647, 0.7552133],
                              [0.3836432, 0.91477764],
                              [0.31375453, 0.9247505],
                              [0.27881047, 0.883953],
                              [0.3026023, 0.8168627]] ,
 [[0.56133837, 0.62012666],
  [0.594052, 0.8567541]]   , [[0.5092937, 0.7252947],
                              [0.6661711, 0.7262009]],
 [[0.7315987, 0.85312754],
  [0.74275094, 0.7262009]], [[0.7241636, 0.6101538],
                             [0.7271376, 0.60743403]],
 [[0.9122679, 0.6799638],
  [0.9159853, 0.6600181]

In [117]:
random_data_point = next(iter(validation.shuffle(200_000).take(1)))
print(random_data_point["strokes"])
print(preprocess_strokes(random_data_point["strokes"]))
# filepath = random_data_point["filepath"].numpy().decode("ascii")
# os.system(f"inkmlviewer {filepath}")

<tf.RaggedTensor [[[230.0, 149.0],
  [232.0, 156.0],
  [232.0, 157.0],
  [233.0, 158.0],
  [233.0, 160.0],
  [234.0, 162.0],
  [234.0, 164.0],
  [235.0, 166.0],
  [235.0, 169.0],
  [236.0, 172.0],
  [236.0, 174.0],
  [237.0, 177.0],
  [237.0, 180.0],
  [237.0, 183.0],
  [237.0, 185.0],
  [238.0, 188.0],
  [238.0, 191.0],
  [238.0, 193.0],
  [239.0, 195.0],
  [239.0, 198.0],
  [239.0, 201.0],
  [240.0, 204.0],
  [240.0, 207.0],
  [240.0, 210.0],
  [240.0, 213.0],
  [241.0, 217.0],
  [241.0, 220.0],
  [241.0, 223.0],
  [241.0, 225.0],
  [241.0, 228.0],
  [241.0, 230.0],
  [241.0, 232.0],
  [241.0, 234.0],
  [241.0, 236.0],
  [241.0, 237.0],
  [241.0, 238.0],
  [241.0, 239.0],
  [241.0, 240.0],
  [241.0, 241.0],
  [241.0, 242.0],
  [241.0, 243.0],
  [241.0, 244.0],
  [241.0, 245.0],
  [241.0, 246.0],
  [241.0, 247.0],
  [241.0, 248.0],
  [241.0, 249.0],
  [241.0, 248.0],
  [241.0, 246.0],
  [240.0, 245.0],
  [240.0, 244.0],
  [240.0, 242.0],
  [239.0, 241.0],
  [239.0, 239.0],
  [239.0, 2

2024-12-16 14:34:32.804781: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 13310734807813386391
2024-12-16 14:34:32.804805: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 13868516638080132147
2024-12-16 14:34:32.804809: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 11636958988054840477
2024-12-16 14:34:32.804813: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 187067494142092949
2024-12-16 14:34:32.804816: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 7235724127813958695
2024-12-16 14:34:32.804821: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 1422916744017125842
2024-12-16 14:34:32.804824: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv i

InvalidArgumentError: Graph execution error:

Detected at node map_1/while/while/cond_1/ArgMax defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/jeshwinprince/Programming/crohme/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/jeshwinprince/Programming/crohme/.venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/jeshwinprince/Programming/crohme/.venv/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/jeshwinprince/Programming/crohme/.venv/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/usr/lib/python3.12/asyncio/base_events.py", line 1987, in _run_once

  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run

  Fil

### Preprocessing datasets

Now, we can go through our datasets a preprocess all the data. We will need both our vectorizer and our stroke preprocessor together. Since we are going with an encoder-decoder model, we need input data for the decoder as well, which should be the desired output, but just missing the last token, and with the start token added in front.

In [103]:
def preprocess_data(data):
    input_strokes = preprocess_strokes(data["strokes"])
    ground_truth = vectorizer([data["ground_truth"]])
    decoder_input = ground_truth[0][:-1]
    decoder_output = ground_truth[0][1:]
    return (input_strokes, decoder_input), decoder_output

In [107]:
for data in test.shuffle(200_000).take(5).map(preprocess_data):
    # pp_data = preprocess_data(data)
    # pp_data = data
    print("Decoder input:", token_to_latex(data[0][1]))
    print("True value:", token_to_latex(data[1]))

Decoder input: <START>1+t
True value: 1+t<END>
Decoder input: <START>\frac{6}{\sqrt{60}}
True value: \frac{6}{\sqrt{60}}<END>
Decoder input: <START>\frac{s}{3}
True value: \frac{s}{3}<END>
Decoder input: <START>\log(x-x^{\prime})
True value: \log(x-x^{\prime})<END>


2024-12-16 14:33:11.328215: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at reduction_ops_common.h:147 : INVALID_ARGUMENT: Invalid reduction dimension (0 for input with 0 dimension(s)


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_3_device_/job:localhost/replica:0/task:0/device:CPU:0}} Error in user-defined function passed to MapDataset:487 transformation with iterator: Iterator::Root::Prefetch::Map: Invalid reduction dimension (0 for input with 0 dimension(s)
	 [[{{function_node map_1_while_while_cond_1_true_19302_tfg_region_specialized_map_1_while_while_cond_1_tfg_inlined_map_1_while_while_1_0}}{{node map_1/while/while/cond_1/Max_tfg_inlined_map_1/while/while/cond_1_tfg_inlined_map_1/while/while_1_0}}]] [Op:IteratorGetNext] name: 

In [ ]:
test = test.map(preprocess_data).shuffle(200_000).batch(32).prefetch(tf.data.AUTOTUNE)
train = train.map(preprocess_data).shuffle(200_000).batch(32).prefetch(tf.data.AUTOTUNE)
validation = (
    validation.map(preprocess_data)
    .shuffle(200_000)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)

Let's test this out to make sure it worked!

In [28]:
print(next(iter(validation.take(1))))

## Model Architecture

My model is an encoder-decoder architecture, with a CNN for the encoder, a feedforward network to get to the latent space, and a LSTM RNN for the decoder.

### Encoder

In [ ]:
input_strokes = layers.Input(shape=(64, 64, 2))
x = layers.Conv2D(64, kernel_size=(3, 3), padding="same")(input_strokes)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(128, kernel_size=(3, 3), padding="same")(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(256, kernel_size=(3, 3), padding="same")(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(512, kernel_size=(3, 3), padding="same")(x)
x = layers.MaxPooling2D((2, 2))(x)

latent_space = layers.Dense(1024, activation="relu")(x)
latent_space = layers.Dense(512, activation="relu")(latent_space)
latent_space = layers.Dense(512, activation="relu")(latent_space)
latent_space = layers.Dense(512, activation="relu")(latent_space)
latent_space = layers.Dense(512, activation="relu")(latent_space)

### Decoder

In [ ]:
latent_space_h = layers.Dense(256, activation="relu")(latent_space)
latent_space_c = layers.Dense(256, activation="relu")(latent_space)

decoder_input = layers.Input(shape=(None,))
decoder_embedding = layers.Embedding(input_dim=vectorizer.vocabulary_size(), output_dim=128)(
    decoder_input
)
decoder_lstm = layers.LSTM(128, return_sequences=True)
decoder_output = decoder_lstm(
    decoder_embedding, initial_state=[latent_space_h, latent_space_c]
)

### Output

In [ ]:
output = layers.Dense(vectorizer.vocabulary_size(), activation="softmax")(decoder_output)
model = keras.Model([input_strokes, decoder_input], output)
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

## Training the Model

Now we can finally train our model! We have a ton of training and validation data to use. We'll also save the history so we can get a graph of the change in loss over each epoch.

In [ ]:
history = model.fit(
    train,
    validation_data=validation,
    epochs=20,
    verbose=1,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
        tf.keras.callbacks.ModelCheckpoint("model_checkpoint.h5", save_best_only=True),
    ],
)